In [27]:
import pandas as pd

In [28]:
df = pd.read_csv('./stock_data.csv')
df

,Date,TSLA,PSNY,LCID,GOOG,AAPL,MSFT,UAL,AAL,DAL,CVX,XOM,SHEL
0,1/2/24,248.42,2.21,4.15,139.56,185.64,370.87,40.72,13.44,40.45,149.48,102.36,65.91
1,12/29/23,248.48,2.26,4.21,140.93,192.53,376.04,41.26,13.74,40.23,149.16,99.98,65.80
2,12/28/23,253.18,2.36,4.26,141.28,193.58,375.28,41.97,13.98,40.63,149.77,100.19,65.45
3,12/27/23,261.44,2.50,4.32,141.44,193.15,374.07,41.73,13.99,40.59,151.91,101.66,66.13
4,12/26/23,256.61,2.49,4.34,142.82,193.05,374.66,42.08,14.11,40.76,152.41,102.14,66.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,1/9/23,119.77,5.55,6.78,88.80,130.15,227.12,43.35,14.61,36.77,175.18,108.47,58.07
244,1/6/23,113.06,5.65,6.36,88.16,129.62,224.93,42.24,14.18,36.03,176.56,110.53,57.25
245,1/5/23,110.34,5.65,6.25,86.77,125.02,222.31,41.24,13.99,35.23,175.24,109.21,55.48
246,1/4/23,113.64,5.49,6.48,88.71,126.36,229.10,39.72,13.59,34.39,172.14,106.82,55.43


In [29]:
# Convert from wide to long format
df_long = pd.melt(df, id_vars=['Date'], var_name='Stock', value_name='Price')

# Parse dates
df_long['Date'] = pd.to_datetime(df_long['Date'])
# Sort by date
df_long = df_long.sort_values(by='Date')

df_long


,Date,Stock,Price
1487,2023-01-03,MSFT,239.58
2727,2023-01-03,XOM,106.51
2479,2023-01-03,CVX,173.99
2231,2023-01-03,DAL,32.61
1983,2023-01-03,AAL,12.74
...,...,...,...
2232,2024-01-02,CVX,149.48
1488,2024-01-02,UAL,40.72
1736,2024-01-02,AAL,13.44
2728,2024-01-02,SHEL,65.91


In [30]:
# Find start price for each stock
start_price = df_long.groupby('Stock')['Price'].first()

# Create column "Value" with percent change in price from start price
df_long = df_long.join(start_price, on='Stock', rsuffix='_start')
df_long['Value'] = (df_long['Price'] - df_long['Price_start']) / df_long['Price_start']
df_long


,Date,Stock,Price,Price_start,Value
1487,2023-01-03,MSFT,239.58,239.58,0.000000
2727,2023-01-03,XOM,106.51,106.51,0.000000
2479,2023-01-03,CVX,173.99,173.99,0.000000
2231,2023-01-03,DAL,32.61,32.61,0.000000
1983,2023-01-03,AAL,12.74,12.74,0.000000
...,...,...,...,...,...
2232,2024-01-02,CVX,149.48,173.99,-0.140870
1488,2024-01-02,UAL,40.72,37.21,0.094329
1736,2024-01-02,AAL,13.44,12.74,0.054945
2728,2024-01-02,SHEL,65.91,55.98,0.177385


In [31]:
rename_dict = {
    'AAPL': 'Tech A',
    'GOOG': 'Tech B',
    'MSFT': 'Tech C',
    'TSLA': 'Car A',  
    'PSNY': 'Car B',
    'LCID': 'Car C',
    'SHEL': 'Gas A',
    'XOM': 'Gas B',
    'CVX': 'Gas C',
    'UAL': 'Airline A',
    'AAL': 'Airline B',
    'DAL': 'Airline C'
}
df_long['name'] = df_long['Stock'].map(rename_dict)
df_long['industry'] = df_long['name'].str.split().str[0]
df_long


,Date,Stock,Price,Price_start,Value,name,industry
1487,2023-01-03,MSFT,239.58,239.58,0.000000,Tech C,Tech
2727,2023-01-03,XOM,106.51,106.51,0.000000,Gas B,Gas
2479,2023-01-03,CVX,173.99,173.99,0.000000,Gas C,Gas
2231,2023-01-03,DAL,32.61,32.61,0.000000,Airline C,Airline
1983,2023-01-03,AAL,12.74,12.74,0.000000,Airline B,Airline
...,...,...,...,...,...,...,...
2232,2024-01-02,CVX,149.48,173.99,-0.140870,Gas C,Gas
1488,2024-01-02,UAL,40.72,37.21,0.094329,Airline A,Airline
1736,2024-01-02,AAL,13.44,12.74,0.054945,Airline B,Airline
2728,2024-01-02,SHEL,65.91,55.98,0.177385,Gas A,Gas


In [32]:
# Sort by "name", then by "Date"
df_long = df_long.sort_values(by=['name', 'Date'])

In [33]:
df_long.to_csv('./clean_stocks.csv', index=False)